In [25]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
import re

In [26]:
load_dotenv()  # take environment variables from .env file

True

In [27]:
model = ChatOpenAI()

In [28]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    evaluation_metric: int
    raw_evaluation: str

In [29]:
def create_outline(state: BlogState) -> BlogState:
    #fetch title 
    title = state['title']

    # Call llm gen outline
    prompt = f"Create a detailed outline for a blog post on a topic - {title}"
    outline = model.invoke(prompt).content

    # Update state
    state['outline'] = outline
    return state

In [30]:
def create_blog(state: BlogState) -> BlogState:
    title = state['title']
    #fetch outline 
    outline = state['outline']

    prompt = f"Write a comprehensive blog post based on the following outline: {outline} for the topic - {title}"
    content = model.invoke(prompt).content

    #Update state
    state['content'] = content
    return state

In [31]:
def evaluate_blog(state: BlogState) -> BlogState:
    content = state['content']
    outline = state['outline']

    prompt = f"""Evaluate the following blog content based on how well it adheres to the outline provided.
    
Rate it on a scale of 1 to 10.

IMPORTANT: Return ONLY a single number between 1 and 10. Do not include any explanation, text, or additional characters. Just the number.

Outline: {outline}

Content: {content}

Rating (number only):"""
    
    evaluation = model.invoke(prompt).content
    state['raw_evaluation'] = evaluation
    return state

In [33]:
def validate_evaluation(state: BlogState) -> BlogState:
    raw_evaluation = state['raw_evaluation']
    
    # First, try to find valid ratings (1-10 as complete words)
    # Pattern matches: "10" OR single digits 1-9, with word boundaries
    numbers = re.findall(r'\b(10|[1-9])\b', raw_evaluation.strip())
    
    if numbers:
        rating = int(numbers[0])
        state['evaluation_metric'] = rating
    else:
        # Fallback: try to extract any digit sequence
        all_digits = re.findall(r'\d+', raw_evaluation.strip())
        if all_digits:
            rating = int(all_digits[0])
            # Clamp to valid range (1-10)
            rating = max(1, min(10, rating))
            state['evaluation_metric'] = rating
        else:
            # If extraction fails completely, store the raw response
            print(f"Warning: Could not extract rating. Storing raw response.")
            state['evaluation_metric'] = raw_evaluation
    
    return state

In [34]:
graph = StateGraph(BlogState)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate_blog)
graph.add_node('validate_evaluation', validate_evaluation)

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', 'validate_evaluation')
graph.add_edge('validate_evaluation', END)

workflow = graph.compile()

In [35]:
initial_state = {
    'title': 'Rise of AI in India'
}

In [36]:
final_state = workflow.invoke(initial_state)

In [37]:
print(final_state['outline'])

I. Introduction
    A. Definition of AI 
    B. Significance of AI in India
    C. Overview of the rise of AI in India 

II. Historical Perspective of AI in India 
    A. Brief history of AI development in India 
    B. Key players and organizations involved in AI research and development 
    C. Milestones in AI technology adoption in India 

III. Current Landscape of AI in India 
    A. Growth of AI startups in India 
    B. Adoption of AI in various sectors such as healthcare, education, finance, etc.
    C. Challenges facing the AI industry in India 

IV. Government Initiatives and Policies to Promote AI in India 
    A. Introduction of National AI Strategy 
    B. Investments in AI research and development 
    C. Collaboration between government and private sector in AI development 

V. Impact of AI on Indian Economy and Job Market 
    A. Positive impact of AI on productivity and efficiency 
    B. Concerns about job displacement due to AI automation 
    C. Steps taken to reski

In [38]:
print(final_state['content'])

Artificial Intelligence, or AI, has become an integral part of our modern world, revolutionizing various industries and sectors with its ability to analyze vast amounts of data, learn from patterns, and make decisions. In India, AI is gaining momentum as a key technology that has the potential to transform the economy and society. This blog post delves into the rise of AI in India, exploring its historical perspective, current landscape, government initiatives, impact on the economy and job market, future prospects, and the need for coordinated efforts to harness its full potential.

I. Introduction

A. Definition of AI 
Artificial Intelligence refers to the simulation of human intelligence processes by machines, including learning, reasoning, and self-correction.

B. Significance of AI in India
AI is significant in India due to its potential to drive innovation, increase productivity, and solve complex challenges across various sectors.

C. Overview of the rise of AI in India
India ha

In [39]:
print(final_state['evaluation_metric'])

9
